In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
alpha = 2
n = 100  # number of time steps
for seed in range(10):
    series = []
    rng = np.random.default_rng(seed)
    rngs = rng.normal(0, 1, size=n)
    for _ in range(n):
        if len(series)>1:
            mu = series[-1]
        else:
            mu = 0
        mu = alpha * np.sum(rngs[:len(series)])
        series.append(mu)
        
    # Plotting
    plt.figure(figsize=(10, 3))
    plt.plot(series, drawstyle='steps-post', marker='o')
    plt.grid(True)
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy

# Parameters
n_points = 100
branch_point = 70
t = np.linspace(0, 4 * np.pi, n_points)

# Deterministic sine wave up to the branching point
pre_branch = np.sin(t[:branch_point])

# Two possible future trajectories after the branch
trajectory_a = np.sin(t[branch_point:])
# Get value at the end of pre-branch (i.e., value before branching)
last_val = pre_branch[-1]

# Use a sine with phase, but adjust amplitude and offset to match last_val
future_t = t[branch_point:]  # future time steps
raw_b = np.cos(future_t + np.pi / 2)

# Scale and shift to start at the same value as pre_branch end
offset = last_val - raw_b[0]
trajectory_b = raw_b + offset

# Combine into full possible trajectories
full_a = np.concatenate((pre_branch, trajectory_a))
full_b = np.concatenate((pre_branch, trajectory_b))

# Plotting
plt.plot(full_a, label='Trajectory A')
plt.plot(full_b, label='Trajectory B', linestyle='--')
plt.axvline(x=branch_point, color='red', linestyle=':', label='Branch Point')
plt.legend()
plt.title("Time Series with Branching")
plt.xlabel("Time Step")
plt.ylabel("Value")
plt.show()

# ---- Entropy Calculation ----
# Two outcomes at the branch point with equal probability
p = np.array([0.5, 0.5])  # probabilities of A and B
H = entropy(p, base=2)  # Shannon entropy in bits

print(f"Entropy at the branching point: {H:.3f} bits")

In [ ]:
import numpy as np
from scipy.stats import entropy
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Parameters
n = 100  # total number of time steps
branch_point = 70
p = 0.7  # Probability of choosing trajectory A
noise_std = 0.1
n_samples = 1000

# Time vectors
t = np.linspace(0, 4 * np.pi, n)
future_t = t[branch_point:]

# Deterministic sine wave pre-branch
pre_branch = np.sin(t[:branch_point])

# Trajectories with internal Gaussian noise
def noisy_trajectory(base_func, noise_std, seed=None):
    rng = np.random.default_rng(seed)
    return base_func + rng.normal(0, noise_std, size=base_func.shape)

# Base trajectories (without noise)
traj_a_base = np.sin(future_t)
# Get value at the end of pre-branch (i.e., value before branching)
last_val = pre_branch[-1]

# Use a sine with phase, but adjust amplitude and offset to match last_val
future_t = t[branch_point:]  # future time steps
raw_b = np.cos(future_t + np.pi / 2)

# Scale and shift to start at the same value as pre_branch end
offset = last_val - raw_b[0]
traj_b_base = raw_b + offset

# Generate samples for entropy estimation
samples_a = np.array([noisy_trajectory(traj_a_base, noise_std, seed=i) for i in range(n_samples)])
samples_b = np.array([noisy_trajectory(traj_b_base, noise_std, seed=1000 + i) for i in range(n_samples)])

# Estimate entropy using PCA-based dimensionality reduction
def estimate_entropy_pca(samples, n_components=3, bins=30):
    reduced = PCA(n_components=n_components).fit_transform(samples)
    hist, _ = np.histogramdd(reduced, bins=bins, density=True)
    hist = hist[hist > 0]  # remove zeros to avoid log(0)
    return entropy(hist.flatten(), base=2)

# Estimate entropy of A and B
H_A = estimate_entropy_pca(samples_a)
H_B = estimate_entropy_pca(samples_b)

# Entropy of branch choice
H_branch = entropy([p, 1 - p], base=2)

# Full conditional entropy
H_cond = H_branch + p * H_A + (1 - p) * H_B

# Print results
print(f"Entropy of Trajectory A: {H_A:.3f} bits")
print(f"Entropy of Trajectory B: {H_B:.3f} bits")
print(f"Entropy of Branch Choice: {H_branch:.3f} bits")
print(f"Total Conditional Entropy H(Future | Past): {H_cond:.3f} bits")

# Plotting the pre-branch and post-branch trajectories
plt.figure(figsize=(10, 5))

# Plot pre-branch
plt.plot(t[:branch_point], pre_branch, label="Pre-Branch (Sine Wave)", color='blue')

# Plot trajectory A and B after the branch
plt.plot(future_t, traj_a_base, label="Trajectory A (Base)", linestyle='--', color='green')
plt.plot(future_t, traj_b_base, label="Trajectory B (Base)", linestyle='--', color='red')

# Adding noisy trajectories to the plot
plt.plot(future_t, samples_a[2], color='green', alpha=0.5, label="Trajectory A (Sample 1)")
plt.plot(future_t, samples_b[2], color='red', alpha=0.5, label="Trajectory B (Sample 1)")

# Labels and title
plt.title("Time Series with Branching and Noise")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.grid(True)

# Show plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
n = 100  # number of time steps
p = 0.01  # probability of flipping to 1
state = 0
series = []

for _ in range(n):
    if state == 0:
        if np.random.rand() < p:
            state = 1
    series.append(state)

# Plotting
plt.figure(figsize=(10, 3))
plt.plot(series, drawstyle='steps-post', marker='o')
plt.title(f"Sticky Binary Time Series (p = {p})")
plt.xlabel("Time")
plt.ylabel("State")
plt.ylim(-0.1, 1.1)
plt.grid(True)
plt.show()